In [1]:
import requests
from io import BytesIO
from PIL import Image

# 익스프레스 프로젝트를 모듈화하기
1. 모듈화 방법 자세히 살펴보기
2. 사용자 정보 관련 기능을 모듈화하기
3. 설정 파일 만들기
4. UI 라이브러리로 웹 문서 예쁘게 꾸미기(생략)

# 모듈화 방법 자세히 살펴보기

In [2]:
%%writefile ./src/user1.js
exports.getUser = function() {
    return {id:'test01',name:'donggyu'};
}

exports.group = {id:'group01',name:'friend'};


Writing ./src/user1.js


In [10]:
%%writefile ./src/module_test1.js
var user1 = require('./user1');

function showUser() {
    return user1.getUser().name + ', ' + user1.group.name;
}
console.log('User Info -> ' + showUser());


Overwriting ./src/module_test1.js


In [11]:
!node ./src/module_test1.js

User Info -> donggyu, friend


In [5]:
%%writefile ./src/user2.js
var user = {
    getUser: function() {
        return {id:'test01',name:'donggyu'};
    },
    group: {id:'group01',name:'friend'}
}

module.exports = user;

Writing ./src/user2.js


In [12]:
%%writefile ./src/module_test2.js
var user = require('./user2');

function showUser() {
    return user.getUser().name + ', ' + user.group.name;
}
console.log('User Info : ' , showUser());

Overwriting ./src/module_test2.js


In [14]:
!node ./src/module_test2.js

User Info :  donggyu, friend


In [15]:
%%writefile ./src/user3.js
module.exports = function() {
    return {id:'test01',name:'donggyu'};
};


Writing ./src/user3.js


In [16]:
%%writefile ./src/module_test3.js
var user = require('./user3');

function showUser() {
    return user().name + ', ' + 'No Group';
}

console.log('User Info : ' + showUser());

Writing ./src/module_test3.js


In [17]:
!node ./src/module_test3.js

User Info : donggyu, No Group


In [20]:
%%writefile ./src/module_test4.js
var require = function(path) {
    var exports = {};
    exports.getUser = function() {
        return {id:'test01',name:'donggyu'};
    };
    exports.group = {id:'group01',name:'friend'};    
    
    return exports;
}

var user = require('...');

function showUser() {
    return user.getUser().name + ', ' + user.group.name;
}

console.log('User Info : ' + showUser());

Overwriting ./src/module_test4.js


In [21]:
!node ./src/module_test4.js

User Info : donggyu, friend


In [22]:
%%writefile ./src/user4.js
exports.printUser = function() {
    console.log('user name is donggyu.');
};

Writing ./src/user4.js


In [26]:
%%writefile ./src/module_test5.js
var printUser = require('./user4').printUser;

printUser();

Overwriting ./src/module_test5.js


In [27]:
!node ./src/module_test5.js

user name is donggyu.


In [28]:
%%writefile ./src/user5.js
function User(id, name) {
    this.id = id;
    this.name = name;
}

User.prototype.getUser = function(){
    return {id:this.id,name:this.name};
};

User.prototype.group = {id:'group01',name:'friend'};

User.prototype.printUser = function(){
    console.log('user name : ' + this.name + ', group : ' + this.group.name);
}

module.exports = new User('test01', 'donggyu');


Writing ./src/user5.js


In [29]:
%%writefile ./src/module_test6.js
var user = require('./user5');

user.printUser();


Writing ./src/module_test6.js


In [30]:
!node ./src/module_test6.js

user name : donggyu, group : friend


In [31]:
%%writefile ./src/user6.js
function User(id, name) {
    this.id = id;
    this.name = name;
}

User.prototype.getUser = function(){
    return {id:this.id,name:this.name};
};

User.prototype.group = {id:'group01',name:'friend'};

User.prototype.printUser = function(){
    console.log('user name : ' + this.name + ', group : ' + this.group.name);
}

module.exports = User;


Writing ./src/user6.js


In [32]:
%%writefile ./src/module_test7.js
var User = require('./user6');

var user = new User('test01', 'donggyu');

user.printUser();

Writing ./src/module_test7.js


In [33]:
!node ./src/module_test7.js

user name : donggyu, group : friend


# 사용자 정보 관련 기능을 모듈화하기

In [34]:
%%writefile ./src/database/user_schema.js

var crypto = require('crypto');

var Schema = {};

Schema.createSchema = function(mongoose) {
    console.log('createSchema 호출됨.');
    
    var UserSchema = mongoose.Schema({
        id: {type:String, required:true, unique:true,'default':''},
        hashed_password: {type:String, required:true,'default':''},
        salt:{type:String,required:true},
        name: {type:String, index:'hashed','default':''},
        age: {type:Number, 'default':-1},
        created_at: {type:Date,index:{unique:false},'default':Date.now()},
        updated_at:{type:Date,index:{unique:false},'default':Date.now()}
    });
    console.log('UserSchema 정의함.');

    UserSchema
        .virtual('password')
        .set(function(password) {
            this.salt = this.makeSalt();
            this.hashed_password = this.encryptPassword(password);
            console.log('virtual password 저장됨 : ' + this.hashed_pasword);
        })
        //.get()

    UserSchema.method('encryptPassword', function(plainText, inSalt){
        if(inSalt){
            return crypto.createHmac('sha1',inSalt).update(plainText).digest('hex');
        } else {
            return crypto.createHmac('sha1', this.salt).update(plainText).digest('hex');
        }
    })

    UserSchema.method('makeSalt', function() {
        return Math.round((new Date().valueOf() * Math.random())) + '';
    });

    UserSchema.method('authenticate', function(plainText,inSalt,hashed_password) {
        if(inSalt) //salt가 넘어온 경우
        {
            console.log('authenticate 호출됨.');
            return this.encryptPassword(plainText, inSalt) === hashed_password;
        } else {
            console.log('authenticate 호출됨.');
            return this.encryptPassword(plainText) === hashed_password;
        }
    })

    //함수 등록(이후 모델 객체에서 사용가능)
    UserSchema.static('findById', function(id, callback) {
        return this.find({id:id}, callback);        
    });

    /*UserSchema.statics.findById = function(id, callback) {
        return this.find({id:id}, callback);
    }*/

    UserSchema.static('findAll', function(callback){
        return this.find({}, callback);        
    })
    
    return UserSchema;
}

module.exports = Schema;

Writing ./src/database/user_schema.js


In [35]:
%%writefile ./src/app11.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

// 암호화 모듈
var crypto = require('crypto');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;
var UserSchema;
var UserModel

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017/local';
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
            
        //UserSchema 별도의 모듈파일(user_schema)로 분리함.
        createUserSchema(database);
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
}

function createUserSchema(database) {
    //database의 속성으로 넣어주기.
    database.UserSchema = require('./database/user_schema').createSchema(mongoose);
    
    database.UserModel = mongoose.model('users3', database.UserSchema);
    console.log('UserModel 정의함.');
    
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));



var router = express.Router();

router.route('/process/adduser').post(function(req,res){
    console.log('/process/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
})

router.route('/process/login').post(function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
})

router.route('/process/listuser').get(function(req,res){
    console.log('/process/listuser 라우팅 함수 호출됨.');
    if(database) {
        UserModel.findAll(function(err, results){
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
            if(results){
                console.dir(results);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write("<h3>사용자 리스트</h3>");
                res.write("<div><ul>");
                for(var i=0; i<results.length; i++){
                    var curId = results[i]._doc.id;
                    var curName = results[i]._doc.name;
                    res.write("    <li>#" + i + "-> " + curId + ", " + curName +
                             "</li>");
                }
                res.write("</ul></div>");
                res.end();
            }else {
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>조회된 사용자 없음.</h1>');
                res.end();
                return;
            }
        })
    }else{
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결안됨.</h1>');
        res.end();
        return;
    }
})

app.use('/', router);

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    UserModel.findById(id, function(err, result) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('아이디 %s로 검색됨.');
        if(result.length > 0){
            var user = new UserModel({id:id})
            var authenticated = user.authenticate(password, result[0],_doc.salt, result[0]._doc.hashed_password);
                
            if(authenticated) {
                console.log('비밀번호 일치함.');
                callback(null, result);
            } else {
                console.log('비밀번호 일치하지 않음.');
                callback(null, null);
            }
        }else{
            console.log('아이디 일치하는 사용자 없음.');
            callback(null, null);
        }
    });
    
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
}

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Writing ./src/app11.js


## 라우팅 함수 모듈화

In [36]:
%%writefile ./src/routes/user.js
var login = function(req,res){
    console.log('/process/login 라우팅 함수 호출됨.');
        
    var paramId = req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword);    
    
    var database = req.app.get('database');
    if (database) {
        authUser(database, paramId, paramPassword, function(err, docs) {
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>에러 발생</h1>');
                res.end();
                return;
            }
                    
            if(docs){
                console.dir(docs);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 로그인 성공</h1>');
                res.write('<div><p>사용자 : ' + docs[0].name + '</p></div>')
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
                return;
            }else{
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
        })    
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;    
    }
}

var adduser = function(req,res){
    console.log('/process/adduser 라우팅 함수 호출됨.');
    var paramId=req.body.id || req.query.id;
    var paramPassword = req.body.password || req.query.password;
    var paramName = req.body.name || req.query.name;
        
    var database = req.app.get('database');
    console.log('요청 파라미터 : ' + paramId + ', ' + paramPassword + ', '+paramName);
    if(database) {
        addUser(database, paramId, paramPassword, paramName, function(err, result){
            if(err) throw err;
            if(result){
                console.dir(result);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 성공</h1>');
                res.write('<div><p>사용자 : ' + paramName + '</p></div>');
                res.write('<br><br><a href="/public/login.html">다시 로그인하기</a>');
                res.end();
            } else {
                console.log('에러 발생');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 추가 안됨</h1>');
                res.end();    
            }
        });
    }else {
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결 안됨.</h1>');
        res.end();
        return;  
    }   
}

var listuser = function(req,res){
    console.log('/process/listuser 라우팅 함수 호출됨.');
    
    var database = req.app.get('database');
    if(database) {
        database.UserModel.findAll(function(err, results){
            if(err){
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>사용자 데이터 조회 안됨.</h1>');
                res.end();
                return;
            }
            if(results){
                console.dir(results);
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write("<h3>사용자 리스트</h3>");
                res.write("<div><ul>");
                for(var i=0; i<results.length; i++){
                    var curId = results[i]._doc.id;
                    var curName = results[i]._doc.name;
                    res.write("    <li>#" + i + "-> " + curId + ", " + curName +
                             "</li>");
                }
                res.write("</ul></div>");
                res.end();
            }else {
                console.log('에러 발생.');
                res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
                res.write('<h1>조회된 사용자 없음.</h1>');
                res.end();
                return;
            }
        })
    }else{
        console.log('에러 발생.');
        res.writeHead(200, {"Content-Type":"text/html;charset=utf-8"});
        res.write('<h1>데이터베이스 연결안됨.</h1>');
        res.end();
        return;
    }
}

var authUser = function(database, id, password, callback) {
    console.log('authUser 호출됨 : ' + id + ', ' + password);
    
    database.UserModel.findById(id, function(err, result) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('아이디 %s로 검색됨.');
        if(result.length > 0){
            var user = new UserModel({id:id})
            var authenticated = user.authenticate(password, result[0],_doc.salt, result[0]._doc.hashed_password);
                
            if(authenticated) {
                console.log('비밀번호 일치함.');
                callback(null, result);
            } else {
                console.log('비밀번호 일치하지 않음.');
                callback(null, null);
            }
        }else{
            console.log('아이디 일치하는 사용자 없음.');
            callback(null, null);
        }
    });
    
}

var addUser = function(database, id, password, name, callback) {
    console.log('addUuser 호출됨 : ' + id + ', ' + password + ', '+ name);
    
    var user = new database.UserModel({"id":id,"password":password,"name":name});
    
    user.save(function(err) {
        if(err) {
            callback(err, null);
            return;
        }
        console.log('사용자 데이터 추가함.');
        callback(null, user);
    });
}

module.exports.login = login;
module.exports.adduser = adduser;
module.exports.listuser = listuser;

Writing ./src/routes/user.js


In [37]:
%%writefile ./src/app12.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

var user = require('./routes/user');

// 암호화 모듈
var crypto = require('crypto');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;

function connectDB() {
    var databaseUrl = 'mongodb://localhost:27017/local';
    
    mongoose.Promise = global.Promise;
    mongoose.connect(databaseUrl);
    database = mongoose.connection;
    
    database.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + databaseUrl);
            
        //UserSchema 별도의 모듈파일(user_schema)로 분리함.
        createUserSchema(database);
    });
    
    database.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
    
    app.set('database', database);
}

function createUserSchema(database) {
    //database의 속성으로 넣어주기.
    database.UserSchema = require('./database/user_schema').createSchema(mongoose);
    
    database.UserModel = mongoose.model('users3', database.UserSchema);
    console.log('UserModel 정의함.');
}

var app = express();

app.set('port', process.env.PORT || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));

var router = express.Router();

router.route('/process/adduser').post(user.adduser);

router.route('/process/login').post(user.login);

router.route('/process/listuser').get(user.listuser);

app.use('/', router);

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    connectDB();
})

Writing ./src/app12.js


# 설정 파일 만들기

In [43]:
%%writefile ./src/config.js
module.exports = {
    server_port: 3000,
    db_url: 'mongodb://localhost:27017/local',
    db_schemas: [
        {file:'./user_schema', collection:'users3', schemaName:'UserSchema', modelName:'UserModel'}
    ],
    route_info: [
        //===== User =====//
        {file:'./user', path:'/process/login', method:'login', type:'post'}					// user.login 
        ,{file:'./user', path:'/process/adduser', method:'adduser', type:'post'}				// user.adduser 
        ,{file:'./user', path:'/process/listuser', method:'listuser', type:'post'}			// user.listuser 
    ]
}

Overwriting ./src/config.js


In [48]:
%%writefile ./src/database/database_loader.js
var mongoose = require('mongoose');

var database = {};

database.init = function(app, config){
    console.log('init 호출됨.');
    
    connect(app, config);
}

function connect(app, config){
    console.log('connect 호출됨.');
    
    mongoose.Promise = global.Promise;
    mongoose.connect(config.db_url);
    database.db = mongoose.connection;
    
    database.db.on('open', function() {
        console.log('데이터베이스에 연결됨 : ' + config.db_url);
            
        createSchema(app, config);
    });
    
    database.db.on('disconnected', function(){
        console.log('데이터베이스 연결 끊어짐.');    
    })
    
    database.db.on('error', console.error.bind(console, 'mongoose 연결 에러.'));
    
}

function createSchema(app, config){
    console.log('설정의 DB 스키마 수 : ' + config.db_schemas.length);
    for(var i =0; i<config.db_schemas.legnth; i++){
        var curItem = config.db_schemas[i];
        
        var curSchema = require(curItem.file).createSchema(mongoose);
        console.log('%s 모듈을 이용해 스키마 생성함.', curItem.file);
        
        var curModel = mongoose.model(curItem.collection, curSchema);
        console.log('%s 컬렉션을 위해 모델 정의함.', curItem.collection);
        
        database[curItem.schemaName] = curSchema;
        database[curItem.modelName] = curModel;
        console.log('스키마 [%s], 모델 [%s] 생성됨.', curItem.schemaName, curItem.modelName);
    }
    app.set('database', database);
}

module.exports = database;

Overwriting ./src/database/database_loader.js


In [46]:
%%writefile ./src/routes/route_loader.js
var route_loader = {};

var config = require('../config');

route_loader.init = function(app, router){
    console.log('route_loader.init 호출됨.');
    
    initRoutes(app, router);
}

function initRoutes(app, router) {
    console.log('initRoutes 호출됨.');
    
    for(var i=0; i<config.route_info.length; i++){
        var curItem = config.route_info[i];
        
        var curModule = require(curItem.file);
        if(curItem.type == 'get'){
            router.route(curItem.path).get(curModule[curItem.method]);
        }else if (curItem.type == 'post') {
            router.route(curItem.path).post(curModule[curItem.method]);
        }else {
            console.log('라우팅 함수의 타입을 알 수 없습니다.: '+curItem.type);
        }
    }
    
    app.use('/', router);
}

module.exports = route_loader;

Overwriting ./src/routes/route_loader.js


In [45]:
%%writefile ./src/app13.js
var express = require('express')
,http = require('http')
,path = require('path');

var bodyParser = require('body-parser') //추가 설치 필요
,static = require('serve-static')
,cookieParser = require('cookie-parser')
,expressSession = require('express-session')

// 에러 핸들러 모듈 사용
var expressErrorHandler = require('express-error-handler');

var user = require('./routes/user');

var config = require('./config');

var database_loader = require('./database/database_loader');
var route_loader = require('./routes/route_loader');

// 암호화 모듈
var crypto = require('crypto');

// mongoose 모듈 사용
var mongoose = require('mongoose');

var database;

function connectDB() {
    //
    app.set('database', database);
}

function createUserSchema(database) {
    //database의 속성으로 넣어주기.
    database.UserSchema = require('./database/user_schema').createSchema(mongoose);
    
    database.UserModel = mongoose.model('users3', database.UserSchema);
    console.log('UserModel 정의함.');
}

var app = express();

console.log('config.server_port -> ' + config.server_port);
app.set('port', config.server_port || 3000);
app.use('/public', static(path.join(__dirname, 'public'))); //middleware

//외장 모듈 미들웨어 등록(body로 들어가는 파라미터를 처리하기 위함)
app.use(bodyParser.urlencoded({extended:false}));
app.use(bodyParser.json());

app.use(cookieParser());
app.use(expressSession({
    secret:'my key',
    resave:true,
    saveUninitialized:true
}));

route_loader.init(app, express.Router());

// 404 에러 페이지 처리
var errorHandler = expressErrorHandler({
    static: {
        '404': './public/404.html'
    }
});

app.use( expressErrorHandler.httpError(404) );
app.use( errorHandler );

//app.get 은 app.set으로 설정한 속성의 값을 가져오는 것
var server = http.createServer(app).listen(app.get('port'), function(){
    console.log('익스프레스로 웹 서버를 실행함 : ' + app.get('port'));
    
    //서버 실행되고 나서 데이터베이스 실행한다. (데이터베이스 먼저 실행해도 됨)
    database_loader.init(app, config);
})

Overwriting ./src/app13.js


# UI 라이브러리로 웹 문서 예쁘게 꾸미기(생략)